# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

Connecting cbroz@dss-db.datajoint.io:3306


DataJoint connection (connected) cbroz@dss-db.datajoint.io:3306

Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

### Imports and activation

Importing schema from `adamacs.pipeline` automatically activates relevant schema.

In [2]:
import datajoint as dj
from adamacs.pipeline import subject, session, surgery, scan
from adamacs import utility
from adamacs.ingest import session as isess

Assign easy names for relevant tables

In [3]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())

## Ingesting scan

In [4]:
#temporary step - insert placeholder values
equipment_placeholder = "Equipment"
location_placeholder = "Location"
from adamacs.pipeline import Equipment, Location
Equipment.insert1({'scanner' : equipment_placeholder}, skip_duplicates=True)
Location.insert1({'anatomical_location': location_placeholder}, skip_duplicates=True) 

In [5]:
isess.ingest_session_scan('sess9FB2LN5C')

/var/folders/_9/tzvq__ws5z9gv5s7jvkj570r0000gn/T/ipykernel_14855/3059593151.py:1: UserWarning: 
Skipped existing session row: ('sess9FB2LN5C', <module 'adamacs.schemas.subject' from '/Volumes/GoogleDrive/My Drive/Projects/Bonn/adamacs/adamacs/schemas/subject.py'>, '2022-03-18')
  isess.ingest_session_scan('sess9FB2LN5C')
/var/folders/_9/tzvq__ws5z9gv5s7jvkj570r0000gn/T/ipykernel_14855/3059593151.py:1: UserWarning: 
Skipped existing SessionDirectory: ('sess9FB2LN5C', PosixPath('/Users/cb/Documents/Bonn'))
  isess.ingest_session_scan('sess9FB2LN5C')
/var/folders/_9/tzvq__ws5z9gv5s7jvkj570r0000gn/T/ipykernel_14855/3059593151.py:1: UserWarning: 
Skipped existing SessionUser row: ('sess9FB2LN5C', 1)
  isess.ingest_session_scan('sess9FB2LN5C')
/var/folders/_9/tzvq__ws5z9gv5s7jvkj570r0000gn/T/ipykernel_14855/3059593151.py:1: UserWarning: 
Scan scan9FB2LN5C already inserted. Skipped
  isess.ingest_session_scan('sess9FB2LN5C')
/var/folders/_9/tzvq__ws5z9gv5s7jvkj570r0000gn/T/ipykernel_14855/305

In [10]:
key='scan9FB2LN5C'
(scan.Scan & f'scan_id=\"{key}\"').fetch1('acq_software')

'ScanImage'

In [6]:
scan.ScanInfo.populate()

TypeError: can only concatenate list (not "str") to list

In [7]:
%debug

> /Volumes/GoogleDrive/My Drive/Projects/Bonn/adamacs/adamacs/paths.py(52)get_scan_image_files()
     50     # Folder structure: root / subject / session / .tif (raw)
     51     data_dir = get_imaging_root_data_dir()
---> 52     sess_dir = Path(data_dir + "/" + get_session_dir())
     53 
     54     if not sess_dir.exists():



ipdb>  u


> /Volumes/GoogleDrive/My Drive/Projects/Bonn/element-calcium-imaging/element_calcium_imaging/scan.py(86)get_scan_image_files()
     84     for sk in scan_key:
     85         sess_dir_query = session_with_id.SessionDirectory & f"session_id = '{sk}'"
---> 86     return _linking_module.get_scan_image_files(scan_key)
     87 
     88 



ipdb>  


> /Volumes/GoogleDrive/My Drive/Projects/Bonn/element-calcium-imaging/element_calcium_imaging/scan.py(198)make()
    196             import scanreader
    197             # Read the scan
--> 198             scan_filepaths = get_scan_image_files(key)
    199             scan = scanreader.read_scan(scan_filepaths)
    200 



...

ipdb>  data_dir


['/Users/cb/Documents/Bonn/']


ipdb>  get_session_dir()


*** TypeError: get_session_dir() missing 1 required positional argument: 'session_key'


ipdb>  quit


In [ ]:
key='scan9FB2LN5C'

In [9]:
query = (scan.Scan & f"scan_id = '{key}'").fetch1()
query

{'session_id': 'sess9FB2LN5C',
 'scan_id': 'scan9FB2LN5C',
 'scanner': 'Equipment',
 'acq_software': 'ScanImage',
 'scan_notes': ''}

In [11]:
scan.ScanInfo()

session_id,scan_id,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage)


In [6]:
scan.ScanInfo.Field()

session_id,scan_id,field_idx,px_height height in pixels,px_width width in pixels,um_height height in microns,um_width width in microns,field_x (um) center of field in the motor coordinate system,field_y (um) center of field in the motor coordinate system,field_z (um) relative depth of field,delay_image (ms) delay between the start of the scan and pixels in this field,roi the scanning roi (as recorded in the acquisition software) containing this field - only relevant to mesoscale scans
sess9FB2LN5C,scan9FB2LN5C,0,256,256,202.5,202.5,0.0,0.0,0.0,=BLOB=,None


In [7]:
scan.ScanInfo.ScanFile()

session_id,scan_id,file_path filepath relative to root data directory
sess9FB2LN5C,scan9FB2LN5C,C:\datajoint\DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C\scan9FB2LN5C_DB_WEZ-8701_02027_00001.tif
